In [1]:
import numpy as np
import scipy as sp
import scipy.signal
import plotly.graph_objects as go

In [34]:
class Kuramoto_base:
    def __init__(self):
        self.times = None

    def fit(self):
        self.real_part = None
        self.Pxx_den = None
        
    def plot(self):
        fig = go.Figure(data=go.Scatter(x=self.times, y=self.real_part))
        fig.update_layout(
            title="Signal",
            xaxis_title="Signal",
            yaxis_title="Time",
            font=dict(
                family="Courier New, monospace",
                size=18)
        )
        fig.show()
        
        fig = go.Figure(data=go.Scatter(x=self.f, y=self.Pxx_den))
        fig.update_layout(
            title="PSD",
            xaxis_title="Freq (log scale)",
            yaxis_title="Power (log scale)",
            font=dict(
                family="Courier New, monospace",
                size=18)
        )
        fig.update_xaxes(type="log")
        fig.update_yaxes(type="log")
        fig.show()

In [35]:
class Kuramoto(Kuramoto_base):    
    def __init__(self, freq, K=1, N=500, sigma=1, delta_t=1e-2):
        super().__init__()
        self.N = N
        self.K = K
        self.freq = freq
        self.delta_t = delta_t
        self.times = np.arange(0, 1, self.delta_t)
        self.sigma = sigma
        self.w = np.random.uniform(freq * 0.75, freq * 1.25, 
                                   size=self.N)

    def fit(self):
        phi = np.empty((len(self.times), self.N))
        phi[0] = np.random.uniform(-np.pi, np.pi, size=self.N)
        for t in range(1, len(self.times)):
            phi[t] = self.w * 2 * np.pi
            sin_diffs = np.array([np.sin(phi[t - 1, i] - phi[t - 1, j]) 
                                 for i in range(self.N) for j in range(self.N)])
            phi[t] += self.K / self.N * sin_diffs.sum(axis=0)
            phi[t] += np.random.normal(0, self.sigma, size=self.N)
            phi[t] = phi[t] * self.delta_t + phi[t - 1] 
        
        signal = np.exp(1j*phi)
        self.real_part = signal.mean(axis=1).real 
        self.f, self.Pxx_den = scipy.signal.welch(self.real_part, fs=1 / self.delta_t, nperseg=1e2)

In [36]:
k = Kuramoto(10, K=10, N=500)

In [37]:
k.fit()

In [38]:
k.plot()

In [69]:
class Kuramoto_big(Kuramoto_base):
    def __init__(self, freqs, K=1, sigma=1, N=500):
        super().__init__()
        self.delta_t = 1e-2
        self.times = np.arange(0, 1, self.delta_t)
        self.models = [Kuramoto(freq, K, N, sigma, self.delta_t) for freq in freqs]

    def fit(self):
        self.Pxx_dens = [None] * len(self.models)
        self.real_parts = [None] * len(self.models)
        for i, model in enumerate(self.models):
            model.fit()
            self.Pxx_dens[i] = model.Pxx_den
            self.real_parts[i] = model.real_part
        self.Pxx_dens = np.array(self.Pxx_dens)
        self.real_parts = np.array(self.real_parts)
        self.Pxx_den = self.Pxx_dens.mean(axis=0)
        self.real_part = self.real_parts.mean(axis=0)
        self.f = self.models[0].f

In [66]:
model = Kuramoto_big([10] * 10 + [15] * 5 + [25] * 5)

In [67]:
model.fit()

In [72]:
model.plot()

In [71]:
model.f = model.models[0].f